In [2]:
import torch
from neural_network.neural_network import NeuralNetwork  # Adjust the import according to your project structure
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [1]:
%run chess_game.py

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-04-12 15:24:21,956 - INFO - Random move due to high epsilon.
2024-04-12 15:24:21,960 - INFO - Turn: White, Move: b2b4, Reward: 1, NN Move: False
2024-04-12 15:24:21,961 - INFO - Current White Score = 1, Current Black Score = 0

2024-04-12 15:24:21,962 - INFO - Random move due to high epsilon.
2024-04-12 15:24:21,964 - INFO - Turn: Black, Move: g7g5, Reward: 1, NN Move: False
2024-04-12 15:24:21,967 - INFO - Current White Score = 1, Current Black Score = 1

2024-04-12 15:24:21,969 - INFO - Random move due to high epsilon.
2024-04-12 15:24:21,971 - INFO - Turn: White, Move: c2c3, Reward: 1, NN Move: False
2024-04-12 15:24:21,974 - INFO - Current White Score = 2, Current Black Score = 1

2024-04-12 15:24:21,976 - INFO - Random move due to high epsilon.
2024-04-12 15:24:21,980 - INFO - Turn: Black, Move: c7c6, Reward: 1, NN Move: False
2024-04-12 15:24:21,981 - INFO - Current White Score = 2, Current Black Score = 2

2024-04-12 15:24:21,984 - INFO - Random move due to high epsilon.
20

: 